In [1]:
import numpy as np

from confetti.explainer.utils import *
import config as cfg
import confetti.CAM.class_activation_map as cam
from confetti.explainer.confetti_explainer import CONFETTI
from pathlib import Path

In [25]:
dataset = 'NATOPS'

model_path = str(cfg.TRAINED_MODELS_DIR / dataset / f"{dataset}_resnet.keras")

# load training data and model
X_train, X_test, y_train, y_test = load_data(dataset, one_hot=False)
model = keras.models.load_model(model_path)

# load samples
sample_file = f"{cfg.DATA_DIR}/{dataset}_samples.csv"
X_samples, y_samples = load_multivariate_ts_from_csv(sample_file)

#Compute CAM weights
training_weights = cam.compute_weights_cam(model=model, X_data=X_train, dataset=dataset,
                                                   save_weights=False, data_type='training')

#Create CONFETTI explainer
ce = CONFETTI(model_path=model_path,
              X_test=X_samples,
              reference_data=X_train,
              weights=training_weights)

#Define where to save the counterfactuals
ce_dir = Path.cwd()

#Generate counterfactuals
ces_naive, ces_optimized = ce.parallelized_counterfactual_generator(
                ce_dir,
                save_counterfactuals=False,
                processes=6,
                alpha=0.5,
                theta=0.75,
                verbose=False,
            )


Probability target is 0.75 and the CE is 0.7878276705741882

Compiled modules for significant speedup can not be used!
https://pymoo.org/installation.html#installation

To disable this warning:
from pymoo.config import Config
Config.warnings['not_compiled'] = False



In [26]:
from benchmark.evaluations.evaluator import Evaluator

og_labels = np.argmax(model.predict(X_samples),axis=1)
ev = Evaluator()
metrics, summary = ev.evaluate_results(model=model,
                                       explainer="Confetti",
                                       dataset="NATOPS",
                                       counterfactuals=ces_optimized,
                                       sample=X_samples,
                                       og_labels=og_labels,
                                       training_data=X_train,
                                       timesteps=51,
                                       channels=24,
                                       alpha=False,
                                       param_config=0.75)

summary

No counterfactual found for instance 0 in dataset. Skipping...
No counterfactual found for instance 1 in dataset. Skipping...
No counterfactual found for instance 2 in dataset. Skipping...
No counterfactual found for instance 3 in dataset. Skipping...
No counterfactual found for instance 4 in dataset. Skipping...
No counterfactual found for instance 5 in dataset. Skipping...
No counterfactual found for instance 6 in dataset. Skipping...
No counterfactual found for instance 7 in dataset. Skipping...
No counterfactual found for instance 8 in dataset. Skipping...
No counterfactual found for instance 10 in dataset. Skipping...
No counterfactual found for instance 11 in dataset. Skipping...
No counterfactual found for instance 12 in dataset. Skipping...
No counterfactual found for instance 13 in dataset. Skipping...
No counterfactual found for instance 14 in dataset. Skipping...
No counterfactual found for instance 15 in dataset. Skipping...
No counterfactual found for instance 16 in datase

,Explainer,Dataset,Coverage,Sparsity,Confidence,Validity,Proximity L1,Proximity L2,Proximity DTW,yNN
0,Confetti,NATOPS,5.555556,0.919118,0.907845,1.0,45.445621,5.878509,5.878509,1.0


In [ ]:
'''import config as cfg

# Dictionary to store dataset name → (timesteps, features)
dataset_shapes = {}

for dataset in cfg.DATASETS:
    X_training, X_testing, y_training, y_testing = load_data(dataset, one_hot=False)

    timesteps = X_training.shape[1]
    features = X_training.shape[2]

    dataset_shapes[dataset] = (timesteps, features)

# Optional: print or return the dictionary
print(dataset_shapes)'''

In [ ]:
'''from confetti.explainer.utils import convert_string_to_array

alphas = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0]
thetas = [0.55, 0.65, 0.75, 0.85, 0.95]
#Load conterfactuals saved in cfg.RESULTS_DIR, and obtain the labels of the counterfactuals
for dataset in cfg.DATASETS:
    model = cfg.TRAINED_MODELS_DIR / dataset / f"{dataset}_fcn.keras"
    ce_dir = cfg.RESULTS_DIR / dataset
    for alpha in alphas:
        ces_naive_df = pd.read_csv(ce_dir / f"confetti_naive_{dataset}_fcn_alpha_{alpha}.csv")
        timesteps = dataset_shapes[dataset][0]
        features = dataset_shapes[dataset][1]
        ces_naive_df['Solution'] = ces_naive_df['Solution'].apply(lambda s: convert_string_to_array(s, timesteps, features))
        ces_naive_df['CE Label'] = ces_naive_df['Solution'].apply(lambda x: np.argmax(model.predict(x.reshape(1, timesteps, channels)), axis=1)[0])

'''